# Assignment using Greedy Algorithms

#### Here I will implement various greedy approaches to task assignment based on a data frame of costs, for my purposes I assumed that rows will be assigned to columns, but the inverse can be accomplished by just transposing the data frame.

In [2]:
import pandas as pd

In [3]:
costmatrix=pd.read_excel('AssignmentProblem10by10.xlsx')

In [4]:
costmatrix

,A,B,C,D,E,F,G,H,I,J
1,53,82,50,97,44,58,54,19,88,59
2,30,16,45,27,56,28,23,11,10,89
3,30,16,45,27,56,28,23,11,10,89
4,64,35,39,48,43,94,48,96,40,94
5,36,74,11,88,3,75,85,84,23,22
6,35,71,59,71,63,79,87,10,58,89
7,87,71,59,86,7,86,15,95,48,18
8,42,4,62,84,40,48,84,84,75,60
9,24,95,68,41,46,68,97,33,43,97
10,49,69,41,69,33,45,87,36,93,96


### Simple greedy assignment, moving from left to right

In [46]:
def greedy(df):
    """
    Takes a cost matrix and greedily assigns the cheapest available row to each column from left to right.
    Assumes that each row can be assigned to one column and vice versa
    Returns a dictionary with each column and the row that was assigned to it, as well as the total assignment cost
    """
    soln={}
    objfun=0
    for i in df.columns:
        soln.update({i:df.drop(list(soln.values()))[i][df[i]==min(df.drop(list(soln.values()))[i])].index.tolist()[0]})
        objfun+=df[i].loc[soln[i]]
    return soln,objfun

### Absolute Minimum Assignment

In [47]:
def findMin(df):
    """
    Takes a cost matrix and scans through all possible assignments to find the absolute lowest cost match for one worker
    and one job.
    Returns the minimum cost and a single entry dictionary with the column:row assignment pair.
    """
    locs = {}
    current_min = 9999999
    for i in df.columns:
        if min(df[i])<current_min:
            current_min = min(df[i])
            locs = {}
            locs.update({i:df[i][df[i]==min(df[i])].index.tolist()[0]})
    return current_min, locs

In [21]:
def trueMinsFirst(df):
    """
    Instead of working sequentially from left to right like the previous algorithms, this algorithm finds the absolute
    minimum cost assignment in the data frame, makes that assignment and then repeats with the tasks and remaining people
    until there are none remaining.
    """
    soln = {}
    cost = 0
    for i in df.columns:
        assignment = findMin(df.drop(soln.values()).drop(soln.keys(),axis=1))
        soln.update(assignment[1])
        cost+=assignment[0]
    return soln, cost

### Swap Assignment

In [25]:
def swapAssignment(df):
    """
    Takes a cost matrix and greedily assigns the cheapest available row to each column from left to right.
    Assumes that each row can be assigned to one column and vice versa
    After each assignment after the first, it checks if swapping the last assignment with the previous one will
        improve the objective function value
    Returns a dictionary with each column and the row that was assigned to it, as well as the total assignment cost
    """
    soln={}
    objfun=0
    for i in df.columns:
        soln.update({i:df.drop(list(soln.values()))[i][df[i]==min(df.drop(list(soln.values()))[i])].index.tolist()[0]})
        if len(soln)>1 and (df.loc[soln[i],i]+df.loc[soln[previous],previous])>(df.loc[soln[previous],i]+df.loc[soln[i],previous]):
            objfun-=df.loc[soln[previous],previous]
            temp = soln[previous]
            soln.update({previous:soln[i],i:temp})
            objfun+=df.loc[soln[previous],previous]
        
        objfun+=df[i].loc[soln[i]]
        previous=i
    return soln,objfun
    

##### Now to test the algorithms on our 10x10 cost matrix:

In [50]:
results = {}

In [54]:
r1 = greedy(costmatrix)
results.update({'left to right':r1[1]})
print(r1)

({'A': 9, 'B': 8, 'C': 5, 'D': 2, 'E': 7, 'F': 3, 'G': 4, 'H': 6, 'I': 1, 'J': 10}, 343)


In [55]:
r2 = trueMinsFirst(costmatrix)
results.update({'true mins first':r2[1]})
print(r2)

({'E': 5, 'B': 8, 'H': 6, 'I': 2, 'G': 7, 'A': 9, 'D': 3, 'C': 4, 'F': 10, 'J': 1}, 236)


In [56]:
r3 = swapAssignment(costmatrix)
results.update({'swap':r3[1]})
print(r3)

({'A': 9, 'B': 8, 'C': 5, 'D': 2, 'E': 7, 'F': 3, 'G': 4, 'H': 1, 'I': 6, 'J': 10}, 322)


In [57]:
results

{'left to right': 343, 'true mins first': 236, 'swap': 322}

### Moving on to 100x100 cost matrix

##### Need to clean it first...

In [30]:
bigcostmatrix = pd.read_csv('AssignmentProblemTestData100.txt',delim_whitespace=True,header=None)


In [31]:
bigcostmatrix.head(16)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,52,89,40,77,89,14,9,77,92,77.0,52.0,53.0,96.0
1,96,92,76,33,81,92,84,36,81,47.0,55.0,87.0,35.0
2,31,71,6,20,8,10,75,54,50,12.0,38.0,5.0,20.0
3,93,70,63,95,96,61,53,35,25,60.0,64.0,42.0,46.0
4,68,20,61,53,61,28,86,16,51,32.0,39.0,19.0,28.0
5,82,31,99,2,30,7,23,53,12,55.0,60.0,75.0,6.0
6,8,9,42,45,80,42,77,42,45,13.0,57.0,62.0,66.0
7,73,21,94,36,10,100,29,46,69,NaN,NaN,NaN,NaN
8,20,17,80,96,14,43,4,69,5,29.0,59.0,18.0,61.0
9,22,82,35,36,8,7,22,15,83,27.0,33.0,27.0,7.0


In [32]:
bigcostmatrix.shape

(800, 13)

In [33]:
bigcostmatrix.loc[0:7]

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,52,89,40,77,89,14,9,77,92,77.0,52.0,53.0,96.0
1,96,92,76,33,81,92,84,36,81,47.0,55.0,87.0,35.0
2,31,71,6,20,8,10,75,54,50,12.0,38.0,5.0,20.0
3,93,70,63,95,96,61,53,35,25,60.0,64.0,42.0,46.0
4,68,20,61,53,61,28,86,16,51,32.0,39.0,19.0,28.0
5,82,31,99,2,30,7,23,53,12,55.0,60.0,75.0,6.0
6,8,9,42,45,80,42,77,42,45,13.0,57.0,62.0,66.0
7,73,21,94,36,10,100,29,46,69,NaN,NaN,NaN,NaN


In [34]:
test = pd.concat([bigcostmatrix.loc[0],bigcostmatrix.loc[1]])
list(test)

[52.0,
 89.0,
 40.0,
 77.0,
 89.0,
 14.0,
 9.0,
 77.0,
 92.0,
 77.0,
 52.0,
 53.0,
 96.0,
 96.0,
 92.0,
 76.0,
 33.0,
 81.0,
 92.0,
 84.0,
 36.0,
 81.0,
 47.0,
 55.0,
 87.0,
 35.0]

In [35]:
i = 0
fixed = []
while i < bigcostmatrix.shape[0]-1:
    fixed.append(list(pd.concat([bigcostmatrix.loc[i],bigcostmatrix.loc[i+1],bigcostmatrix.loc[i+2],bigcostmatrix.loc[i+3],
                            bigcostmatrix.loc[i+4],bigcostmatrix.loc[i+5],bigcostmatrix.loc[i+6],bigcostmatrix.loc[i+7]])))
    i+=8

[[52.0, 89.0, 40.0, 77.0, 89.0, 14.0, 9.0, 77.0, 92.0, 77.0, 52.0, 53.0, 96.0, 96.0, 92.0, 76.0, 33.0, 81.0, 92.0, 84.0, 36.0, 81.0, 47.0, 55.0, 87.0, 35.0, 31.0, 71.0, 6.0, 20.0, 8.0, 10.0, 75.0, 54.0, 50.0, 12.0, 38.0, 5.0, 20.0, 93.0, 70.0, 63.0, 95.0, 96.0, 61.0, 53.0, 35.0, 25.0, 60.0, 64.0, 42.0, 46.0, 68.0, 20.0, 61.0, 53.0, 61.0, 28.0, 86.0, 16.0, 51.0, 32.0, 39.0, 19.0, 28.0, 82.0, 31.0, 99.0, 2.0, 30.0, 7.0, 23.0, 53.0, 12.0, 55.0, 60.0, 75.0, 6.0, 8.0, 9.0, 42.0, 45.0, 80.0, 42.0, 77.0, 42.0, 45.0, 13.0, 57.0, 62.0, 66.0, 73.0, 21.0, 94.0, 36.0, 10.0, 100.0, 29.0, 46.0, 69.0, nan, nan, nan, nan], [20.0, 17.0, 80.0, 96.0, 14.0, 43.0, 4.0, 69.0, 5.0, 29.0, 59.0, 18.0, 61.0, 22.0, 82.0, 35.0, 36.0, 8.0, 7.0, 22.0, 15.0, 83.0, 27.0, 33.0, 27.0, 7.0, 7.0, 64.0, 83.0, 32.0, 70.0, 53.0, 92.0, 79.0, 33.0, 46.0, 60.0, 34.0, 43.0, 58.0, 6.0, 9.0, 54.0, 66.0, 66.0, 95.0, 62.0, 95.0, 91.0, 63.0, 74.0, 34.0, 88.0, 40.0, 22.0, 59.0, 43.0, 86.0, 34.0, 31.0, 58.0, 37.0, 35.0, 95.0, 53.0, 63

In [36]:
fixedbigcostmatrix = pd.DataFrame(fixed)
print(fixedbigcostmatrix.shape)
fixedbigcostmatrix.head()

(100, 104)


,0,1,2,3,4,5,6,7,8,9,...,94,95,96,97,98,99,100,101,102,103
0,52.0,89.0,40.0,77.0,89.0,14.0,9.0,77.0,92.0,77.0,...,36.0,10.0,100.0,29.0,46.0,69.0,NaN,NaN,NaN,NaN
1,20.0,17.0,80.0,96.0,14.0,43.0,4.0,69.0,5.0,29.0,...,9.0,57.0,31.0,76.0,88.0,92.0,NaN,NaN,NaN,NaN
2,17.0,34.0,50.0,77.0,71.0,31.0,74.0,26.0,100.0,20.0,...,42.0,98.0,32.0,76.0,39.0,31.0,NaN,NaN,NaN,NaN
3,55.0,63.0,85.0,35.0,31.0,46.0,51.0,70.0,21.0,24.0,...,80.0,30.0,17.0,36.0,47.0,49.0,NaN,NaN,NaN,NaN
4,70.0,13.0,14.0,26.0,49.0,80.0,24.0,48.0,31.0,42.0,...,63.0,78.0,17.0,50.0,85.0,67.0,NaN,NaN,NaN,NaN


In [37]:
fixedbigcostmatrix.drop(100,axis=1,inplace=True)
fixedbigcostmatrix.drop(101,axis=1,inplace=True)
fixedbigcostmatrix.drop(102,axis=1,inplace=True)
fixedbigcostmatrix.drop(103,axis=1,inplace=True)

In [38]:
fixedbigcostmatrix.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,52.0,89.0,40.0,77.0,89.0,14.0,9.0,77.0,92.0,77.0,...,66.0,73.0,21.0,94.0,36.0,10.0,100.0,29.0,46.0,69.0
1,20.0,17.0,80.0,96.0,14.0,43.0,4.0,69.0,5.0,29.0,...,15.0,49.0,89.0,34.0,9.0,57.0,31.0,76.0,88.0,92.0
2,17.0,34.0,50.0,77.0,71.0,31.0,74.0,26.0,100.0,20.0,...,63.0,50.0,95.0,52.0,42.0,98.0,32.0,76.0,39.0,31.0
3,55.0,63.0,85.0,35.0,31.0,46.0,51.0,70.0,21.0,24.0,...,75.0,13.0,20.0,58.0,80.0,30.0,17.0,36.0,47.0,49.0
4,70.0,13.0,14.0,26.0,49.0,80.0,24.0,48.0,31.0,42.0,...,57.0,81.0,15.0,24.0,63.0,78.0,17.0,50.0,85.0,67.0


##### Now to actually test the algorithms:

In [28]:
bigmatrixresults = {}

In [49]:
result1 = greedy(fixedbigcostmatrix)
bigmatrixresults.update({'left to right':result1[1]})
print(result1)

({0: 13, 1: 14, 2: 48, 3: 69, 4: 33, 5: 7, 6: 90, 7: 59, 8: 28, 9: 8, 10: 88, 11: 65, 12: 96, 13: 12, 14: 31, 15: 21, 16: 50, 17: 5, 18: 10, 19: 82, 20: 83, 21: 4, 22: 62, 23: 16, 24: 81, 25: 46, 26: 23, 27: 58, 28: 40, 29: 2, 30: 93, 31: 47, 32: 42, 33: 38, 34: 70, 35: 89, 36: 49, 37: 25, 38: 67, 39: 3, 40: 27, 41: 44, 42: 74, 43: 64, 44: 94, 45: 30, 46: 56, 47: 61, 48: 52, 49: 79, 50: 32, 51: 18, 52: 39, 53: 53, 54: 97, 55: 24, 56: 76, 57: 87, 58: 57, 59: 72, 60: 19, 61: 9, 62: 36, 63: 54, 64: 73, 65: 41, 66: 15, 67: 98, 68: 0, 69: 26, 70: 80, 71: 55, 72: 37, 73: 43, 74: 1, 75: 17, 76: 68, 77: 6, 78: 60, 79: 45, 80: 85, 81: 75, 82: 77, 83: 29, 84: 86, 85: 35, 86: 20, 87: 95, 88: 11, 89: 78, 90: 63, 91: 71, 92: 66, 93: 91, 94: 99, 95: 34, 96: 84, 97: 51, 98: 22, 99: 92}, 553.0)


In [43]:
result2 = trueMinsFirst(fixedbigcostmatrix)
bigmatrixresults.update({'true mins first':result2[1]})
print(result2)

({73: 55, 1: 14, 2: 48, 3: 69, 4: 33, 5: 7, 7: 59, 8: 28, 9: 8, 10: 88, 12: 96, 13: 12, 14: 31, 15: 21, 16: 50, 18: 10, 20: 83, 22: 62, 23: 16, 28: 40, 29: 2, 30: 93, 32: 42, 33: 46, 35: 89, 37: 25, 38: 67, 39: 3, 40: 27, 46: 56, 47: 61, 48: 47, 49: 79, 51: 18, 52: 39, 53: 53, 55: 23, 58: 57, 61: 9, 62: 4, 63: 54, 65: 41, 68: 0, 75: 81, 79: 58, 84: 86, 86: 36, 87: 75, 88: 43, 91: 71, 94: 19, 0: 13, 6: 90, 11: 65, 26: 87, 34: 70, 36: 49, 42: 74, 43: 64, 44: 94, 50: 32, 56: 76, 69: 26, 72: 37, 80: 45, 85: 38, 89: 73, 96: 60, 97: 34, 17: 5, 19: 82, 24: 78, 41: 44, 45: 30, 66: 15, 67: 98, 82: 77, 92: 66, 95: 20, 99: 99, 21: 29, 60: 80, 74: 1, 76: 72, 57: 17, 25: 63, 59: 97, 70: 84, 71: 91, 77: 6, 31: 95, 81: 85, 78: 35, 54: 51, 98: 11, 64: 22, 27: 68, 90: 92, 83: 24, 93: 52}, 522.0)


In [44]:
result3 = swapAssignment(fixedbigcostmatrix)
bigmatrixresults.update({'swap':result3[1]})
print(result3)

({0: 13, 1: 14, 2: 48, 3: 69, 4: 33, 5: 7, 6: 90, 7: 59, 8: 28, 9: 8, 10: 88, 11: 65, 12: 96, 13: 12, 14: 31, 15: 21, 16: 50, 17: 5, 18: 10, 19: 82, 20: 83, 21: 4, 22: 62, 23: 16, 24: 81, 25: 46, 26: 23, 27: 58, 28: 40, 29: 2, 30: 93, 31: 47, 32: 42, 33: 38, 34: 70, 35: 89, 36: 49, 37: 25, 38: 67, 39: 3, 40: 27, 41: 44, 42: 74, 43: 64, 44: 94, 45: 30, 46: 56, 47: 61, 48: 52, 49: 79, 50: 32, 51: 18, 52: 39, 53: 53, 54: 97, 55: 24, 56: 76, 57: 87, 58: 57, 59: 72, 60: 19, 61: 9, 62: 36, 63: 54, 64: 73, 65: 41, 66: 15, 67: 98, 68: 0, 69: 26, 70: 80, 71: 55, 72: 37, 73: 43, 74: 1, 75: 17, 76: 68, 77: 6, 78: 60, 79: 45, 80: 85, 81: 75, 82: 77, 83: 29, 84: 86, 85: 35, 86: 20, 87: 95, 88: 11, 89: 78, 90: 63, 91: 71, 92: 66, 93: 91, 94: 99, 95: 34, 96: 84, 97: 51, 98: 92, 99: 22}, 531.0)


In [45]:
bigmatrixresults

{'left to right': 553.0, 'true mins first': 522.0, 'swap': 531.0}

##### It looks like for both cost matrices, pursuing the best possible assignment first yields the best results, followed by the swapping algorithm, and finally the basic left to right algorithm. This makes sense, as the "true mins first algorithm" has the highest complexity, and the basic left to right algorithm has the least.